In [2]:
# Import Required libraries

# library to handle data in a vectorized manner
import numpy as np 

# library for data analysis
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# library to handle JSON files
import json 

# installing geocoder and converting addresses into latitude and longitude values
!pip install geocoder
import geocoder

# package for getting coordinates
#!pip install geopy
from geopy.geocoders import Nominatim

# library to handle requests
import requests 

# library to parse HTML and XML documents
#!pip install beautifulsoup4
from bs4 import BeautifulSoup 

# tranforming JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

     |████████████████████████████████| 98 kB 9.5 MB/s  eta 0:00:01


In [3]:
# map rendering library
!pip install folium
import folium 

     |████████████████████████████████| 94 kB 3.8 MB/s eta 0:00:011


# Scrapping Bangalore neighborhood data from Wikipedia into DataFrame

In [4]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text

# parsing data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
#list to save neighborhood data
bl_neighborhoods = []
#append neighborhoods to list after parsing data from wiki
for item in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    bl_neighborhoods.append(item.text)
    
#Create Data frame from list
bl_neigh_df = pd.DataFrame({"Neighborhood": bl_neighborhoods })
bl_neigh_df.head()

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"


In [5]:
# drop first 3 entries from data frame since they dont contain neighborhood info
bl_neigh_df.drop([0,1,2],inplace=True)
bl_neigh_df.reset_index(inplace=True)
bl_neigh_df.drop(['index'],axis=1,inplace=True)
bl_neigh_df.head()

,Neighborhood
0,Adugodi
1,"Agara, Bangalore"
2,Ananthnagar
3,Anjanapura
4,Arekere


In [6]:
#Size of DataFrame
bl_neigh_df.shape

(140, 1)

## Get Geographical info for Neighborhoods

In [7]:
# function to get coordinates
def get_latlong(neighborhood):
    lat_long = None
    while(lat_long is None):
        neigh = neighborhood + ', Bengaluru, Karnataka'
        g = geocoder.arcgis(neigh)
        lat_long = g.latlng    
    return lat_long

In [8]:
# call the above function to get coordinates
latlong_l = []

for neighborhood in bl_neigh_df["Neighborhood"].tolist():
    lat_long = get_latlong(neighborhood)
    latlong_l.append(lat_long) 

print("Coordinates of neighborhoods:", latlong_l[0:5])

Coordinates of neighborhoods: [[12.944020000000023, 77.60800000000006], [12.842830000000049, 77.48759000000007], [12.954080000000033, 77.54135000000008], [12.858110000000067, 77.55909000000008], [12.885680000000036, 77.59668000000005]]


In [9]:
# create a data frame with lat long info.
df_lat_long = pd.DataFrame(latlong_l, columns=['Latitude', 'Longitude'])
df_lat_long.head()

,Latitude,Longitude
0,12.94402,77.60800
1,12.84283,77.48759
2,12.95408,77.54135
3,12.85811,77.55909
4,12.88568,77.59668


In [10]:
#Merge lat long to neighborhood dataframe
bl_neigh_df['Latitude'] = df_lat_long['Latitude']
bl_neigh_df['Longitude'] = df_lat_long['Longitude']
bl_neigh_df.head()

,Neighborhood,Latitude,Longitude
0,Adugodi,12.94402,77.60800
1,"Agara, Bangalore",12.84283,77.48759
2,Ananthnagar,12.95408,77.54135
3,Anjanapura,12.85811,77.55909
4,Arekere,12.88568,77.59668


## Create map of Bangalore with neighborhoods


In [11]:
# Get coorodinates for Bangalore
address = 'Bengaluru, Karnataka'

geolocator = Nominatim(user_agent="bl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Bengaluru, Karnataka are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of Bengaluru, Karnataka are 12.9791198, 77.5912997.


In [12]:
#Visualize Neigborhoods on Toronto map
# create map of Toronto using latitude and longitude values
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bl_neigh_df['Latitude'], bl_neigh_df['Longitude'], bl_neigh_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore


## Using Foursqaure to explore Bangalore Neighborhoods

In [13]:
#Define Foursquare Credentials and Version
#Marking Fouraqaue credentials Hidden

In [14]:
# @hidden_cell
CLIENT_ID = 'ZOEKN0AVEBVZTL04BWYZP2VWSWKY0ZOS1KJECJMALLLT0P4U' # your Foursquare ID
CLIENT_SECRET = 'PHTC2HZZ5J0TF4XPR1YZPATWCUFMBSRSWFKXA2EMBKAP3PYJ' # your Foursquare Secret
VERSION = '20210123' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [15]:
LIMIT = 100
#function to get near by venues for neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
#Get venues info
bangalore_venues = getNearbyVenues(names=bl_neigh_df['Neighborhood'],
                                   latitudes=bl_neigh_df['Latitude'],
                                   longitudes=bl_neigh_df['Longitude']
                                  )

Adugodi
Agara, Bangalore
Ananthnagar
Anjanapura
Arekere
Austin Town
Babusapalya
Bagalur, Bangalore Urban
Bahubalinagar
Baiyyappanahalli
Baiyyappanahalli Manavarthe Kaval
Banashankari
Banaswadi
Basavanagudi
Basaveshwaranagar
Bellandur
Bengaluru Pete
Bharathnagar
Bilekahalli
Binnamangala Manavarthe Kaval
Binnamangala, Bangalore
Bommanahalli
Bommasandra
Brookefield
BTM Layout
Byatarayanapura
Carmelaram
Chamrajpet
Chandapura
Chickpet
Chikkalasandra
Cooke Town
Cox Town, Bangalore
CV Raman Nagar
Dasarahalli
Devara Jeevanahalli
Devarachikkanahalli
Dollars Colony
Domlur
Ejipura
Electronic City
Fraser Town, Bangalore
G. M. Palya
Gandhi Bazaar
Gandhi Nagar, Bangalore
Gangamma Circle
Ganganagar, Bangalore
Garudacharpalya
Girinagar
Gottigere
Gowdanapalya
Hanumanthanagar, Bengaluru
Hebbagodi
Hebbal
Hoodi
Horamavu
HSR Layout
Hulimavu
Immadihalli
Indiranagar
Ittamadu
Jakkasandra
Jakkur
Jalahalli
Jayanagar, Bangalore
Jayaprakash Nagar, Bangalore
Jeevanbimanagar
Jigani
Kadugodi
Kaggadasapura
Kalasipaly

In [17]:
#Lets see Size of resulting bangalore_venues
bangalore_venues.shape

(7418, 7)

In [18]:
bangalore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adugodi,12.94402,77.608,PVR IMAX,12.934595,77.611321,Movie Theater
1,Adugodi,12.94402,77.608,Tommy Hilfiger,12.934552,77.611347,Clothing Store
2,Adugodi,12.94402,77.608,Koramangala Social,12.935518,77.614097,Lounge
3,Adugodi,12.94402,77.608,Truffles Ice & Spice,12.933443,77.614265,Burger Joint
4,Adugodi,12.94402,77.608,Dyu Art Cafe,12.937289,77.617591,Financial or Legal Service


In [19]:
#number of venues returned for each neighborhood
bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Ananthnagar,30,30,30,30,30,30
Anjanapura,4,4,4,4,4,4
Arekere,81,81,81,81,81,81
Austin Town,100,100,100,100,100,100
BTM Layout,89,89,89,89,89,89
Babusapalya,50,50,50,50,50,50
"Bagalur, Bangalore Urban",10,10,10,10,10,10
Bahubalinagar,23,23,23,23,23,23


In [20]:
#unique categories from all the returned venues
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 238 uniques categories.


## Analyze each Neighborhood

In [21]:
#one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighbourhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
bangalore_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Ramen Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
# @hidden_cell
bangalore_onehot.drop(['Neighborhood'],axis=1,inplace=True)
bangalore_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Ramen Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
bangalore_onehot.shape

(7418, 238)

In [24]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
bangalore_grouped = bangalore_onehot.groupby('Neighbourhood').mean().reset_index()
bangalore_grouped

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Ramen Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adugodi,0.0,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.020000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,

In [25]:
#size of dataframe after grouping
bangalore_grouped.shape

(139, 238)

In [26]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5
for hood in bangalore_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = bangalore_grouped[bangalore_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adugodi----
                venue  freq
0   Indian Restaurant  0.17
1                Café  0.09
2        Dessert Shop  0.05
3         Pizza Place  0.04
4  Chinese Restaurant  0.04


----Ananthnagar----
                  venue  freq
0           Pizza Place  0.10
1           Coffee Shop  0.10
2  Fast Food Restaurant  0.10
3                  Café  0.10
4     Indian Restaurant  0.07


----Anjanapura----
                  venue  freq
0                  Pool  0.25
1           Sports Club  0.25
2           Coffee Shop  0.25
3            Campground  0.25
4  Other Great Outdoors  0.00


----Arekere----
                     venue  freq
0        Indian Restaurant  0.10
1                     Café  0.07
2  South Indian Restaurant  0.06
3         Department Store  0.06
4              Pizza Place  0.05


----Austin Town----
               venue  freq
0  Indian Restaurant  0.10
1              Hotel  0.09
2     Ice Cream Shop  0.05
3                Pub  0.04
4            Brewery  0.04


----BTM Lay


----Jigani----
                  venue  freq
0                   Spa   0.2
1     Indian Restaurant   0.2
2                 Hotel   0.2
3  Gym / Fitness Center   0.2
4          Tech Startup   0.2


----Kadugodi----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.17
1                    Pizza Place  0.17
2           Fast Food Restaurant  0.17
3                 Sandwich Place  0.17
4                    Supermarket  0.17


----Kaggadasapura----
                  venue  freq
0     Indian Restaurant  0.37
1  Fast Food Restaurant  0.09
2      Department Store  0.07
3            Food Truck  0.05
4                  Café  0.05


----Kalasipalyam----
                  venue  freq
0     Indian Restaurant  0.32
1  Fast Food Restaurant  0.05
2                 Hotel  0.04
3        Ice Cream Shop  0.03
4                  Park  0.03


----Kalkere----
               venue  freq
0        Supermarket  0.25
1        Pizza Place  0.25
2  Convenience Store  0.25
3    Warehouse Sto

               venue  freq
0  Indian Restaurant  0.16
1               Café  0.12
2             Bakery  0.12
3      Train Station  0.04
4     Clothing Store  0.04


----Yeswanthpur----
                venue  freq
0       Train Station  0.14
1   Indian Restaurant  0.07
2         Coffee Shop  0.07
3           Multiplex  0.07
4  Miscellaneous Shop  0.07




In [27]:
#create a pandas dataframe with above info
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
#create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = bangalore_grouped['Neighbourhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adugodi,Indian Restaurant,Café,Dessert Shop,Chinese Restaurant,Pizza Place,Ice Cream Shop,Bookstore,Lounge,Gym,Restaurant
1,Ananthnagar,Pizza Place,Fast Food Restaurant,Café,Coffee Shop,Park,Dessert Shop,Indian Restaurant,Fried Chicken Joint,Food Truck,Juice Bar
2,Anjanapura,Campground,Sports Club,Pool,Coffee Shop,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Financial or Legal Service,Field
3,Arekere,Indian Restaurant,Café,Department Store,South Indian Restaurant,Pizza Place,Ice Cream Shop,Fast Food Restaurant,Multiplex,Bakery,Clothing Store
4,Austin Town,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Café,Brewery,Tea Room,Lounge,Chinese Restaurant,Italian Restaurant


## Cluster Neighborhoods

In [29]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#Run k-means to cluster the neighborhood into 5 clusters
kclusters = 5

bangalore_grouped_clustering = bangalore_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 2, 1, 2, 2, 4, 4, 1, 4, 2], dtype=int32)

## new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangalore_merged = bl_neigh_df

# merge bangalore_grouped with bl_neigh_df to add latitude/longitude for each neighborhood
bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

bangalore_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adugodi,12.94402,77.60800,4.0,Indian Restaurant,Café,Dessert Shop,Chinese Restaurant,Pizza Place,Ice Cream Shop,Bookstore,Lounge,Gym,Restaurant
1,"Agara, Bangalore",12.84283,77.48759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ananthnagar,12.95408,77.54135,2.0,Pizza Place,Fast Food Restaurant,Café,Coffee Shop,Park,Dessert Shop,Indian Restaurant,Fried Chicken Joint,Food Truck,Juice Bar
3,Anjanapura,12.85811,77.55909,1.0,Campground,Sports Club,Pool,Coffee Shop,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Financial or Legal Service,Field
4,Arekere,12.88568,77.59668,2.0,Indian Restaurant,Café,Department Store,South Indian Restaurant,Pizza Place,Ice Cream Shop,Fast Food Restaurant,Multiplex,Bakery,Clothing Store


In [31]:
bangalore_merged.dropna(inplace=True)
bangalore_merged['Cluster Labels'] = bangalore_merged['Cluster Labels'].astype(int)
bangalore_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adugodi,12.94402,77.60800,4,Indian Restaurant,Café,Dessert Shop,Chinese Restaurant,Pizza Place,Ice Cream Shop,Bookstore,Lounge,Gym,Restaurant
2,Ananthnagar,12.95408,77.54135,2,Pizza Place,Fast Food Restaurant,Café,Coffee Shop,Park,Dessert Shop,Indian Restaurant,Fried Chicken Joint,Food Truck,Juice Bar
3,Anjanapura,12.85811,77.55909,1,Campground,Sports Club,Pool,Coffee Shop,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Financial or Legal Service,Field
4,Arekere,12.88568,77.59668,2,Indian Restaurant,Café,Department Store,South Indian Restaurant,Pizza Place,Ice Cream Shop,Fast Food Restaurant,Multiplex,Bakery,Clothing Store
5,Austin Town,12.96348,77.61297,2,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Café,Brewery,Tea Room,Lounge,Chinese Restaurant,Italian Restaurant


## Visulaize resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['Latitude'], bangalore_merged['Longitude'], bangalore_merged['Neighborhood'], bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [33]:
#cluster0
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0, bangalore_merged.columns[[0] + list(range(3, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Chamrajpet,0,Indian Restaurant,Fast Food Restaurant,Ice Cream Shop,Coffee Shop,Metro Station,Department Store,Restaurant,Movie Theater,Hotel,Café
28,Chandapura,0,Indian Restaurant,Coffee Shop,Asian Restaurant,Yoga Studio,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market
29,Chickpet,0,Indian Restaurant,Hotel,Coffee Shop,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Breakfast Spot,Theater,Restaurant,Arcade
36,Devarachikkanahalli,0,Indian Restaurant,Pizza Place,Department Store,BBQ Joint,Chinese Restaurant,Dumpling Restaurant,Badminton Court,South Indian Restaurant,Café,Fish Market
55,Horamavu,0,Indian Restaurant,Coffee Shop,Department Store,Badminton Court,Yoga Studio,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
69,Kaggadasapura,0,Indian Restaurant,Fast Food Restaurant,Department Store,Coffee Shop,Café,Food Truck,Pizza Place,Korean Restaurant,Diner,Chinese Restaurant
70,Kalasipalyam,0,Indian Restaurant,Fast Food Restaurant,Hotel,Park,Coffee Shop,Ice Cream Shop,Theater,Restaurant,Movie Theater,Dessert Shop
75,Kengeri,0,Indian Restaurant,Bus Station,Café,Train Station,Grocery Store,Department Store,Arcade,Event Space,Falafel Restaurant,Farmers Market
79,Koppa gate,0,Pizza Place,Spa,Resort,Indian Restaurant,Farmers Market,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant
106,Ramachandrapura,0,Indian Restaurant,Plaza,Ice Cream Shop,Chinese Restaurant,Shopping Mall,Vegetarian / Vegan Restaurant,Indie Movie Theater,Coffee Shop,Fast Food Restaurant,Gym


In [34]:
#cluster1
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1, bangalore_merged.columns[[0] + list(range(3, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Anjanapura,1,Campground,Sports Club,Pool,Coffee Shop,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Financial or Legal Service,Field
7,"Bagalur, Bangalore Urban",1,Pizza Place,Italian Restaurant,Mediterranean Restaurant,Stadium,Sandwich Place,Bakery,North Indian Restaurant,Brewery,Vegetarian / Vegan Restaurant,Coffee Shop
26,Carmelaram,1,Restaurant,Sporting Goods Shop,Soccer Field,Train Station,Bar,Yoga Studio,Flea Market,Fish Market,Financial or Legal Service,Field
71,Kalkere,1,Convenience Store,Pizza Place,Warehouse Store,Supermarket,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Financial or Legal Service,Field
107,Ramagondanahalli,1,ATM,Forest,Dhaba,Gas Station,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Financial or Legal Service,Field
122,Sulikunte,1,Restaurant,Go Kart Track,Flower Shop,Furniture / Home Store,Athletics & Sports,Bar,Café,Flea Market,Fish Market,Financial or Legal Service
132,"Vijayanagar, Bangalore",1,Pizza Place,Grocery Store,Mediterranean Restaurant,Stadium,Sandwich Place,Restaurant,Music Venue,Brewery,Italian Restaurant,Badminton Court


In [35]:
#cluster2
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2, bangalore_merged.columns[[0] + list(range(3, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Ananthnagar,2,Pizza Place,Fast Food Restaurant,Café,Coffee Shop,Park,Dessert Shop,Indian Restaurant,Fried Chicken Joint,Food Truck,Juice Bar
4,Arekere,2,Indian Restaurant,Café,Department Store,South Indian Restaurant,Pizza Place,Ice Cream Shop,Fast Food Restaurant,Multiplex,Bakery,Clothing Store
5,Austin Town,2,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Café,Brewery,Tea Room,Lounge,Chinese Restaurant,Italian Restaurant
9,Baiyyappanahalli,2,Indian Restaurant,Café,Coffee Shop,Pizza Place,Fast Food Restaurant,Department Store,Bakery,Mexican Restaurant,Clothing Store,Pub
10,Baiyyappanahalli Manavarthe Kaval,2,Indian Restaurant,Café,Coffee Shop,Pizza Place,Fast Food Restaurant,Department Store,Bakery,Mexican Restaurant,Clothing Store,Pub
15,Bellandur,2,Indian Restaurant,Café,Fast Food Restaurant,Coffee Shop,Hotel,Lounge,Pizza Place,Ice Cream Shop,Gym,Asian Restaurant
22,Bommasandra,2,Hotel,Sporting Goods Shop,Coffee Shop,Café,Fast Food Restaurant,Pizza Place,Indian Restaurant,History Museum,Historic Site,Fish Market
23,Brookefield,2,Indian Restaurant,Café,Coffee Shop,Pizza Place,Fast Food Restaurant,Sandwich Place,Light Rail Station,Department Store,Convenience Store,Platform
25,Byatarayanapura,2,Café,Coffee Shop,Indian Restaurant,Metro Station,Fast Food Restaurant,Breakfast Spot,Park,Ice Cream Shop,Chinese Restaurant,Seafood Restaurant
38,Domlur,2,Café,Fast Food Restaurant,Indian Restaurant,Restaurant,Coffee Shop,Pizza Place,Cafeteria,Tea Room,Department Store,Middle Eastern Restaurant


In [36]:
#cluster3
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3, bangalore_merged.columns[[0] + list(range(3, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Bharathnagar,3,Ice Cream Shop,Coffee Shop,Pizza Place,Department Store,Dessert Shop,Café,Flea Market,Fish Market,Flower Shop,Electronics Store
125,Thindlu,3,Ice Cream Shop,Dessert Shop,Coffee Shop,Pizza Place,Indian Restaurant,Hookah Bar,Donut Shop,Flea Market,Fish Market,Financial or Legal Service
131,Vidyaranyapura,3,Ice Cream Shop,Coffee Shop,Pizza Place,Department Store,Dessert Shop,Café,Flea Market,Fish Market,Flower Shop,Electronics Store


In [37]:
#cluster4
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 4, bangalore_merged.columns[[0] + list(range(3, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adugodi,4,Indian Restaurant,Café,Dessert Shop,Chinese Restaurant,Pizza Place,Ice Cream Shop,Bookstore,Lounge,Gym,Restaurant
6,Babusapalya,4,Indian Restaurant,BBQ Joint,Korean Restaurant,Ice Cream Shop,Lounge,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Café,Bistro
8,Bahubalinagar,4,Indian Restaurant,Hotel,Ice Cream Shop,Bakery,Intersection,Italian Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Diner,Dessert Shop
11,Banashankari,4,Indian Restaurant,Ice Cream Shop,Café,Fast Food Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Italian Restaurant,Juice Bar,Chinese Restaurant
12,Banaswadi,4,Indian Restaurant,BBQ Joint,Korean Restaurant,Café,Ice Cream Shop,Fast Food Restaurant,Department Store,Pizza Place,Bistro,Bakery
13,Basavanagudi,4,Indian Restaurant,Ice Cream Shop,Café,Fast Food Restaurant,Sandwich Place,Coffee Shop,Breakfast Spot,Juice Bar,Pizza Place,Snack Place
14,Basaveshwaranagar,4,Indian Restaurant,Café,Fast Food Restaurant,Ice Cream Shop,Bakery,Vegetarian / Vegan Restaurant,Pizza Place,Chinese Restaurant,Gym,Mobile Phone Shop
16,Bengaluru Pete,4,Indian Restaurant,Pizza Place,Café,Fast Food Restaurant,Chinese Restaurant,Restaurant,Asian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Hotel
18,Bilekahalli,4,Indian Restaurant,Ice Cream Shop,Bakery,Coffee Shop,Department Store,Café,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Chinese Restaurant
19,Binnamangala Manavarthe Kaval,4,Indian Restaurant,Bakery,Movie Theater,Middle Eastern Restaurant,Café,Park,Tea Room,Food & Drink Shop,Fast Food Restaurant,Mediterranean Restaurant
